# Develop Output Class

In [71]:
class ModelOutput:
    def __init__(self, locals_input):
        # Dictionary with all variables from execute_model
        self.locals_input = locals_input
        # Take locals_input and store variables in different Categories
        self.__filter_locals_input()
        self.__sort_Input()
        self.__sort_Intakes()
        self.__sort_Requirements()
        self.__sort_Production()
        self.__sort_Excretion()
        self.__sort_Digestibility()
        self.__sort_Efficiencies()
        self.__sort_Miscellaneous()


    def __populate_category(self, category_name, group_names, *variable_lists):
        """
        Creates and populates nested dictionaries using lists of variiable names. The variable_lists must be passed
        in the same order as group_names. Use when variables need to be grouped into a dictionary.

        category_name: Name of dictionary to populate
        group_names: Names of the neseted dictionaries to create
        variable_lists: List of variable names, one list for each nested dictionary 
        """ 
        # Check if the Category exists, and create it if it does not
        if not hasattr(self, category_name):
            setattr(self, category_name, {})
        # Populate the category with variables
        for name, var_list in zip(group_names, variable_lists):
            # Create a nested dictionary for the current name if it doesn't exist
            if name not in getattr(self, category_name):
                getattr(self, category_name)[name] = {}
            # Populate the nested dictionary with variables from the corresponding list
            for var in var_list:
                if var in self.locals_input:
                    getattr(self, category_name)[name][var] = self.locals_input.pop(var)


    def __filter_locals_input(self):
        variables_to_remove = ['key', 'value', 'num_value', 'feed_library_df', 
                               'feed_data', 'diet_info_initial', 'diet_data_initial',
                                'AA_list', 'An_data_initial']
        for key in variables_to_remove:
            # Remove values that should be excluded from output
            self.locals_input.pop(key, None)


    def __sort_Input(self):
        setattr(self, 'Inputs', {})
        variables_to_add = ['user_diet', 'animal_input', 'equation_selection', 'coeff_dict', 'infusion_input', 'MP_NP_efficiency_input']
        for key in variables_to_add:
            # Add to the Inputs Category
            self.Inputs[key] = self.locals_input[key]
            # Remove so that values are only stored in one place
            self.locals_input.pop(key, None)


    def __sort_Intakes(self):
        setattr(self, 'Intakes', {})
        variables_to_add = ['diet_info', 'infusion_data', 'diet_data', 'An_data']
        for key in variables_to_add:
            self.Intakes[key] = self.locals_input[key]
            self.locals_input.pop(key, None)

        # Name intake groups
        group_names = ['energy', 'protein', 'AA', 'FA', 'rumen_digestable'] 
        # Lists of variables to store
        energy_variables = ['An_MEIn', 'An_NEIn', 'An_NE', 'An_MEIn_approx']
        protein_variables = ['An_MPIn', 'An_MPIn_g']
        AA_variables = ['AA_values', 'Abs_EAA_g', 'Abs_neAA_g', 'Abs_OthAA_g', 'Abs_EAA2b_g']
        FA_variables = []
        rumen_digestable_variables = ['Rum_DigNDFIn', 'Rum_DigStIn']
        # Store variables
        self.__populate_category('Excretion', group_names, energy_variables, protein_variables, AA_variables, FA_variables, rumen_digestable_variables)


    def __sort_Requirements(self):
        # setattr(self, 'Requirements', {})

        # Name digestability groups
        group_names = ['energy', 'protein']
        # Lists of variables to store
        energy_variables = ['Gest_NCPgain_g', 'Gest_NPgain_g', 'Gest_NPuse_g', 'Gest_CPuse_g', 'An_NEmUse_NS', 'An_NEm_Act_Graze', 'An_NEm_Act_Parlor', 'An_NEm_Act_Topo',
                            'An_NEmUse_Act', 'An_NEmUse', 'An_MEmUse', 'Gest_MEuse', 'Trg_Mlk_NEout', 'Trg_Mlk_MEout', 'Trg_MEuse']
        protein_variables = ['An_MPm_g_Trg', 'Body_NPgain_g', 'Body_MPUse_g_Trg', 'Gest_MPUse_g_Trg', 'Trg_Mlk_NP_g', 'Mlk_MPUse_g_Trg', 'An_MPuse_g_Trg', 'Min_MPuse_g',
                             'Diff_MPuse_g', 'Frm_NPgain_g', 'Frm_MPUse_g_Trg', 'Rsrv_NPgain_g', 'Rsrv_MPUse_g_Trg']
        # Store variables
        self.__populate_category('Requirements', group_names, energy_variables, protein_variables)


        ### Store mineral requirements in nested dictionaries ###
        mineral_abbreviations = ['Ca', 'P', 'Mg', 'Na', 'Cl', 'K', 'S', 'Co', 'Cu', 'I', 'Fe', 'Mn', 'Se', 'Zn']
        mineral_variable_lists = [
            ['Ca_Mlk', 'Fe_Ca_m', 'An_Ca_g', 'An_Ca_y', 'An_Ca_l', 'An_Ca_Clf', 'An_Ca_req', 'An_Ca_bal', 'An_Ca_prod'],
            ['Ur_P_m', 'Fe_P_m', 'An_P_m', 'An_P_g', 'An_P_y', 'An_P_l', 'An_P_Clf', 'An_P_req', 'An_P_bal', 'Fe_P_g', 'An_P_prod'],
            ['An_Mg_Clf', 'Ur_Mg_m', 'Fe_Mg_m', 'An_Mg_m', 'An_Mg_g', 'An_Mg_y', 'An_Mg_l', 'An_Mg_req', 'An_Mg_bal', 'An_Mg_prod'],
            ['An_Na_Clf', 'Fe_Na_m', 'An_Na_g', 'An_Na_y', 'An_Na_l', 'An_Na_req', 'An_Na_bal', 'An_Na_prod'],
            ['An_Cl_Clf', 'Fe_Cl_m', 'An_Cl_g', 'An_Cl_y', 'An_Cl_l', 'An_Cl_req', 'An_Cl_bal', 'An_Cl_prod'],
            ['An_K_Clf', 'Ur_K_m', 'Fe_K_m', 'An_K_m', 'An_K_g', 'An_K_y', 'An_K_l', 'An_K_req', 'An_K_bal', 'An_K_prod'],
            ['An_S_req', 'An_S_bal'],
            ['An_Co_req', 'An_Co_bal'],
            ['An_Cu_Clf', 'An_Cu_m', 'An_Cu_g', 'An_Cu_y', 'An_Cu_l', 'An_Cu_req', 'An_Cu_bal', 'An_Cu_prod'],
            ['An_I_req', 'An_I_bal'],
            ['An_Fe_Clf', 'An_Fe_g', 'An_Fe_y', 'An_Fe_l', 'An_Fe_req', 'An_Fe_bal', 'An_Fe_prod'],
            ['An_Mn_Clf', 'An_Mn_m', 'An_Mn_g', 'An_Mn_y', 'An_Mn_l', 'An_Mn_req', 'An_Mn_bal', 'An_Mn_prod'],
            ['An_Se_req', 'An_Se_bal'],
            ['An_Zn_Clf', 'An_Zn_m', 'An_Zn_g', 'An_Zn_y', 'An_Zn_l', 'An_Zn_req', 'An_Zn_bal', 'An_Zn_prod']
        ]
        mineral_requirements = {}
        # NOTE Since all the mineral requirements are nested within mineral_requirements can't use populate_category()
        for abbreviation, variable_list in zip(mineral_abbreviations, mineral_variable_lists):
            mineral_requirements[abbreviation] = {}  # Initialize the dictionary for the current mineral
            for variable in variable_list:
                if variable in self.locals_input:
                    mineral_requirements[abbreviation][variable] = self.locals_input.pop(variable)
        self.Requirements['mineral_requirements'] = mineral_requirements


    def __sort_Production(self):
        # Name production groups
        group_names = ['milk', 'composition', 'gestation', 'MiCP']
        # List variables to store
        milk_variables = ['Trg_NEmilk_Milk', 'Mlk_NP_g', 'Mlk_CP_g', 'Trg_Mlk_Fat' ,'Trg_Mlk_Fat_g', 'Mlk_Fatemp_g', 'Mlk_Fat_g', 'Mlk_Fat', 'Mlk_NP', 'Mlk_Prod_comp',
                          'An_MPavail_Milk_Trg', 'Mlk_NP_MPalow_Trg_g', 'Mlk_Prod_MPalow', 'An_MEavail_Milk', 'Mlk_Prod_NEalow', 'Mlk_Prod', 'MlkNP_Milk', 'MlkFat_Milk', 'MlkNE_Milk', 'Mlk_NEout', 'Mlk_MEout']
        composition_variables = ['CPGain_FrmGain', 'NPGain_FrmGain', 'Frm_Gain', 'Rsrv_Gain', 'Rsrv_Gain_empty', 'NPGain_RsrvGain', 'Rsrv_NPgain',
                                 'Frm_Gain_empty', 'Body_Gain_empty', 'Frm_NPgain', 'Body_NPgain', 'Body_CPgain', 'Body_CPgain_g', 'Rsrv_Fatgain', 'Rsrv_CPgain', 'Rsrv_NEgain', 'An_BWmature_empty', 'Body_Gain']
        gestation_variables = ['Uter_Wtpart', 'Uter_Wt', 'GrUter_Wtpart', 'GrUter_Wt', 'Uter_BWgain', 'GrUter_BWgain', 'Rsrv_MEgain', 'FatGain_FrmGain', 'Frm_Fatgain',
                               'Frm_CPgain', 'Frm_NEgain', 'Frm_MEgain', 'An_MEgain', 'Gest_REgain']
        MiCP_variables = ['RDPIn_MiNmax', 'MiN_Vm', 'Du_MiN_g', 'Du_MiCP_g', 'Du_MiTP_g', 'Du_MiCP', 'Du_idMiCP_g', 'Du_idMiCP', 'Du_idMiTP_g', 'Du_idMiTP']
        # Store variables
        self.__populate_category('Production', group_names, milk_variables, composition_variables, gestation_variables, MiCP_variables)


    def __sort_Excretion(self):
        # Name excretion groups
        group_names = ['fecal', 'urinary', 'gaseous', 'scurf'] 
        # Lists of variables to store
        fecal_variables = ['Fe_rOMend', 'Fe_RUP', 'Fe_RumMiCP', 'Fe_CPend_g', 'Fe_CPend', 'Fe_CP', 'Fe_NPend', 'Fe_NPend_g', 'Fe_MPendUse_g_Trg']
        urinary_variables = ['Ur_Nout_g', 'Ur_DEout', 'Ur_Nend_g', 'Ur_NPend_g', 'Ur_MPendUse_g']
        gaseous_variables = []
        scurf_variables = ['Scrf_CP_g', 'Scrf_NP_g', 'Scrf_MPUse_g_Trg']
        # Store variables
        self.__populate_category('Excretion', group_names, fecal_variables, urinary_variables, gaseous_variables, scurf_variables)


    def __sort_Digestibility(self):
        # Name digestability groups
        group_names = ['rumen', 'TT']
        # Lists of variables to store
        rumen_variables = ['Rum_dcNDF', 'Rum_dcSt']
        TT_variables = []
        # Store variables
        self.__populate_category('Digestibility', group_names, rumen_variables, TT_variables)


    def __sort_Efficiencies(self):
        # Name digestability groups
        group_names = ['energy', 'protein']
        # Lists of variables to store
        energy_variables = ['Kr_ME_RE']
        protein_variables = ['Kg_MP_NP_Trg']
        # Store variables
        self.__populate_category('Efficiencies', group_names, energy_variables, protein_variables)


    def __sort_Miscellaneous(self):
        # These variables need to be given a storage location
        group_names = ['misc']
        # Lists of variables to store
        misc_variables = ['Kb_LateGest_DMIn', 'An_PrePartWklim', 'An_PrePartWkDurat', 'An_DMIn_BW', 'f_mPrt_max', 'mPrt_k_EAA2', 'An_REgain_Calf', 'An_LactDay_MlkPred', 'An_DCADmeq']
        # Store variables
        self.__populate_category('Miscellaneous', group_names, misc_variables)

    
    


# Develop execute_model

Keep a copy of execute_model() here to develop new functions

In [6]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint
from icecream import ic
# use package dir to always use /data folder regardless of where this is executed from
import importlib_resources

path_to_package_data = importlib_resources.files("nasem_dairy.data")

# Read_csv to load required data into env
user_diet_in, animal_input_in, equation_selection_in = nd.read_csv_input(path_to_package_data.joinpath("./input.csv"))

# Load feed library
feed_library_in = pd.read_csv(path_to_package_data.joinpath("NASEM_feed_library.csv"))

# Prepare infusion data (This will be optional as a default dict of 0 is provided to function otherwise)
infusion_custom = nd.read_infusion_input(path_to_package_data.joinpath("infusion_input.csv"))


In [72]:
def dev_run_NASEM_model(user_diet: pd.DataFrame, 
                        animal_input: dict, 
                        equation_selection: dict, 
                        feed_library_df: pd.DataFrame, 
                        coeff_dict: dict = nd.coeff_dict,
                        infusion_input: dict = nd.infusion_dict,
                        MP_NP_efficiency_input: dict = nd.MP_NP_efficiency_dict
                        ):
    ########################################
    # Step 1: Read User Input
    ########################################
    # prevent mutable changes outside of expected scope (especially for Shiny):
    user_diet = user_diet.copy()
    animal_input = animal_input.copy()

    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = fl_get_rows(list_of_feeds, path_to_db)
    feed_data = nd.get_feed_rows_feedlibrary(list_of_feeds, feed_library_df)


    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    # list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')

    feed_data = feed_data.reset_index(names='Feedstuff')

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()

    diet_info_initial.insert(1, 'Fd_DMInp', 
        Fd_DMInp.reindex(
            diet_info_initial['Feedstuff']).values)

    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    # diet_info_initial['Fd_DNDF48'] = 0

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del (list_of_feeds, Fd_DMInp)

    # Check equation_selection to make sure they are integers.
    # This is especially important for Shiny, which may return strings
    # It's important they are correct for if statements below.
    equation_selection_in = equation_selection.copy()
    equation_selection = {}

    for key, value in equation_selection_in.items():
        try:
            num_value = int(value)
            equation_selection[key] = num_value
        except ValueError:
            print(f"Unable to convert '{value}' to an integer for key '{key}'")
    
    del(equation_selection_in)
    Scrf_CP_g = nd.calculate_Scrf_CP_g(animal_input['An_StatePhys'],
                                       animal_input['An_BW'])
    CPGain_FrmGain = nd.calculate_CPGain_FrmGain(animal_input['An_BW'], 
                                                 animal_input['An_BW_mature'])
    NPGain_FrmGain = nd.calculate_NPGain_FrmGain(CPGain_FrmGain,
                                                 nd.coeff_dict)
    Frm_Gain = nd.calculate_Frm_Gain(animal_input['Trg_FrmGain'])
    Rsrv_Gain = nd.calculate_Rsrv_Gain(animal_input['Trg_RsrvGain'])
    Rsrv_Gain_empty = nd.calculate_Rsrv_Gain_empty(Rsrv_Gain)
    NPGain_RsrvGain = nd.calculate_NPGain_RsrvGain(nd.coeff_dict)
    Rsrv_NPgain = nd.calculate_Rsrv_NPgain(NPGain_RsrvGain,
                                           Rsrv_Gain_empty)

    # if animal_input['An_StatePhys'] != 'Lactating Cow':
    #     animal_input['Trg_MilkProd'] = None

    ############################################################################################
    # Start of refactored NASEM 
    ############################################################################################

    ########################################
    # Step 2: DMI Equations
    ########################################
    # pre calculations for DMI:
    ###########################
    # Calculate Target milk net energy
    Trg_NEmilk_Milk = nd.calculate_Trg_NEmilk_Milk(animal_input['Trg_MilkTPp'],
                                                animal_input['Trg_MilkFatp'],
                                                animal_input['Trg_MilkLacp'])
    
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    # Predict DMI for heifers
    Kb_LateGest_DMIn = nd.calculate_Kb_LateGest_DMIn(Dt_NDF)
    An_PrePartWklim = nd.calculate_An_PrePartWklim(animal_input['An_PrePartWk'])
    An_PrePartWkDurat = An_PrePartWklim * 2


    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8:
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['Trg_MilkProd'],
            animal_input['An_BW'],
            animal_input['An_BCS'],
            animal_input['An_LactDay'],
            animal_input['An_Parity_rl'],
            Trg_NEmilk_Milk
            )

    # Individual Heifer DMI Predictions
    elif equation_selection['DMIn_eqn'] in [2, 3, 4, 5, 6, 7]:
        Dt_DMIn_BW_LateGest_i = nd.calculate_Dt_DMIn_BW_LateGest_i(An_PrePartWklim, Kb_LateGest_DMIn, coeff_dict)

        # All the individual DMI predictions require this value
        Dt_DMIn_Heif_LateGestInd = nd.calculate_Dt_DMIn_Heif_LateGestInd(animal_input['An_BW'], Dt_DMIn_BW_LateGest_i)

        if equation_selection['DMIn_eqn'] == 2:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature']), 
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature'])

        if equation_selection['DMIn_eqn'] == 3:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF)

        if equation_selection['DMIn_eqn'] == 4:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW']),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 5:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'], Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'], Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'],Dt_NDFdev_DMI)

        if equation_selection['DMIn_eqn'] == 6:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW']),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 7:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'],Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'],Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'], Dt_NDFdev_DMI)

    # Group Heifer DMI Predictions
    elif equation_selection['DMIn_eqn'] in [12, 13, 14, 15, 16, 17]:
        Dt_DMIn_BW_LateGest_p = nd.calculate_Dt_DMIn_BW_LateGest_p(An_PrePartWkDurat, Kb_LateGest_DMIn, coeff_dict)

        # All group DMI predicitons require this value
        Dt_DMIn_Heif_LateGestPen = nd.calculate_Dt_DMIn_Heif_LateGestPen(animal_input['An_BW'], Dt_DMIn_BW_LateGest_p)

        if equation_selection['DMIn_eqn'] == 12:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature']),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature'])

        if equation_selection['DMIn_eqn'] == 13:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF)

        if equation_selection['DMIn_eqn'] == 14:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW']), 
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 15:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'],Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'], Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'],Dt_NDFdev_DMI)

        if equation_selection['DMIn_eqn'] == 16:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW']), 
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 17:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'],Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'],Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'], Dt_NDFdev_DMI)

    elif equation_selection['DMIn_eqn'] == 10:
        Dt_DMIn_BW_LateGest_i = nd.calculate_Dt_DMIn_BW_LateGest_i(An_PrePartWklim, Kb_LateGest_DMIn, coeff_dict)

        Dt_DMIn_BW_LateGest_p = nd.calculate_Dt_DMIn_BW_LateGest_p(An_PrePartWkDurat, Kb_LateGest_DMIn, coeff_dict)

        if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
            animal_input['DMI'] = min(
                nd.calculate_Dt_DMIn_DryCow1_FarOff(animal_input['An_BW'], Dt_DMIn_BW_LateGest_i),
                nd.calculate_Dt_DMIn_DryCow1_Close(animal_input['An_BW'], Dt_DMIn_BW_LateGest_p)
                )
        else:
            animal_input['DMI'] = nd.calculate_Dt_DMIn_DryCow1_FarOff(animal_input['An_BW'], Dt_DMIn_BW_LateGest_i)

    elif equation_selection['DMIn_eqn'] == 11:
        animal_input['DMI'] = nd.calculate_Dt_DMIn_DryCow2(animal_input['An_BW'], animal_input['An_GestDay'], animal_input['An_GestLength'])

    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: " + str(equation_selection['DMIn_eqn']))

    # Calculated again as part of diet_data, value may change depending on DMIn_eqn selections
    del (Dt_NDF)

    ########################################
    # Step 3: Feed Based Calculations
    ########################################
    # Calculate An_DMIn_BW with the final DMI value; required for calculating diet_data_initial
    An_DMIn_BW = nd.calculate_An_DMIn_BW(animal_input['An_BW'],
                                         animal_input['DMI'])
    Fe_rOMend = nd.calculate_Fe_rOMend(animal_input['DMI'],
                                       coeff_dict)

    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions

    diet_data_initial = nd.calculate_diet_data_initial(diet_info,
                                                       animal_input['DMI'],
                                                       animal_input['An_BW'],
                                                       animal_input['An_StatePhys'],
                                                       An_DMIn_BW,
                                                       animal_input['An_AgeDryFdStart'],
                                                       animal_input['Env_TempCurr'],
                                                       equation_selection['DMIn_eqn'],
                                                       Fe_rOMend,
                                                       coeff_dict)
    # diet_data contains everything starting with "Dt_"

    ########################################
    # Step 4: Infusion Calculations
    ########################################

    infusion_data = nd.calculate_infusion_data(infusion_input, animal_input['DMI'], coeff_dict)

########################################
# # Step 4.1: Uterine Bodyweight Calculations
# ########################################
    Uter_Wtpart = nd.calculate_Uter_Wtpart(animal_input['Fet_BWbrth'], coeff_dict)
    Uter_Wt = nd.calculate_Uter_Wt(animal_input['An_Parity_rl'],
                                   animal_input['An_AgeDay'],
                                   animal_input['An_LactDay'],
                                   animal_input['An_GestDay'],
                                   animal_input['An_GestLength'],
                                   Uter_Wtpart,
                                   coeff_dict)
                                   
    GrUter_Wtpart = nd.calculate_GrUter_Wtpart(animal_input['Fet_BWbrth'], coeff_dict)
    GrUter_Wt = nd.calculate_GrUter_Wt(animal_input['An_GestDay'],
                                       animal_input['An_GestLength'],
                                       Uter_Wt,
                                       GrUter_Wtpart,
                                       coeff_dict)

    Uter_BWgain = nd.calculate_Uter_BWgain(animal_input['An_LactDay'],
                                           animal_input['An_GestDay'],
                                           animal_input['An_GestLength'],
                                           Uter_Wt,
                                           coeff_dict)

    GrUter_BWgain = nd.calculate_GrUter_BWgain(animal_input['An_LactDay'],
                                               animal_input['An_GestDay'],
                                               animal_input['An_GestLength'],
                                               GrUter_Wt,
                                               Uter_BWgain,
                                               coeff_dict)

    ########################################
    # Step 5: Animal Level Calculations
    ########################################
    # Combine Diet and Infusion nutrient supplies
    An_data_initial = nd.calculate_An_data_initial(animal_input,
                                                   diet_data_initial,
                                                   infusion_data,
                                                   equation_selection['Monensin_eqn'],
                                                   GrUter_Wt,
                                                   coeff_dict)

    ########################################
    # Step 6: Rumen Digestion Calculations
    ########################################
    # Rumen Digestability Coefficients
    Rum_dcNDF = nd.calculate_Rum_dcNDF(animal_input['DMI'],
                                       diet_data_initial['Dt_NDFIn'],
                                       diet_data_initial['Dt_StIn'],
                                       diet_data_initial['Dt_CPIn'],
                                       diet_data_initial['Dt_ADFIn'],
                                       diet_data_initial['Dt_ForWet'])

    Rum_dcSt = nd.calculate_Rum_dcSt(animal_input['DMI'],
                                     diet_data_initial['Dt_ForNDF'],
                                     diet_data_initial['Dt_StIn'],
                                     diet_data_initial['Dt_ForWet'])

    # Rumen Digestable Intakes
    Rum_DigNDFIn = nd.calculate_Rum_DigNDFIn(Rum_dcNDF,
                                             diet_data_initial['Dt_NDFIn'])

    Rum_DigStIn = nd.calculate_Rum_DigStIn(Rum_dcSt,
                                           diet_data_initial['Dt_StIn'])

    ########################################
    # Step 7: Microbial Protein Calculations
    ########################################
    if equation_selection['MiN_eqn'] == 1:
        RDPIn_MiNmax = nd.calculate_RDPIn_MiNmax(animal_input['DMI'],
                                                 An_data_initial['An_RDP'],
                                                 An_data_initial['An_RDPIn'])
        
        MiN_Vm = nd.calculate_MiN_Vm(RDPIn_MiNmax, coeff_dict)
        
        Du_MiN_g = nd.calculate_Du_MiN_NRC2021_g(MiN_Vm,
                                                 Rum_DigNDFIn,
                                                 Rum_DigStIn,
                                                 An_data_initial['An_RDPIn_g'],
                                                 coeff_dict)
    elif equation_selection['MiN_eqn'] == 2:
        Du_MiN_g = nd.calculate_Du_MiN_VTln_g(diet_data_initial['Dt_rOMIn'],
                                              diet_data_initial['Dt_ForNDFIn'],
                                              An_data_initial['An_RDPIn'],
                                              Rum_DigStIn,
                                              Rum_DigNDFIn,
                                              coeff_dict)
    elif equation_selection['MiN_eqn'] == 3:
        Du_MiN_g = nd.calculate_Du_MiN_VTnln_g(An_data_initial['An_RDPIn'],
                                               Rum_DigNDFIn,
                                               Rum_DigStIn)
    else:
        raise ValueError(
            f"Invalid MiN_eqn: {equation_selection['MiN_eqn']} was entered. Must choose 1, 2 or 3.")

    Du_MiCP_g = nd.calculate_Du_MiCP_g(Du_MiN_g)
    Du_MiTP_g = nd.calculate_Du_MiTP_g(Du_MiCP_g, coeff_dict)
    Du_MiCP = nd.calculate_Du_MiCP(Du_MiCP_g)
    Du_idMiCP_g = nd.calculate_Du_idMiCP_g(Du_MiCP_g,
                                           coeff_dict)
    Du_idMiCP = nd.calculate_Du_idMiCP(Du_idMiCP_g)

    ########################################
    # Step 7.1: Fe_CP Calculation /  Finish Dt_ and An_ calculations
    ########################################
    # Required to finish Dt_ and An_ calculations
    Fe_RUP = nd.calculate_Fe_RUP(An_data_initial['An_RUPIn'],
                                 infusion_data['InfSI_TPIn'],
                                 An_data_initial['An_idRUPIn'])
    Fe_RumMiCP = nd.calculate_Fe_RumMiCP(Du_MiCP,
                                         Du_idMiCP)
    
    Fe_CPend_g = nd.calculate_Fe_CPend_g(animal_input['An_StatePhys'],
                                         An_data_initial['An_DMIn'],
                                         An_data_initial['An_NDF'],
                                         animal_input['DMI'],
                                         diet_data_initial['Dt_DMIn_ClfLiq'],
                                         equation_selection["NonMilkCP_ClfLiq"])
    
    Fe_CPend = nd.calculate_Fe_CPend(Fe_CPend_g)
    Fe_CP = nd.calculate_Fe_CP(animal_input['An_StatePhys'],
                               diet_data_initial['Dt_CPIn_ClfLiq'],
                               diet_data_initial['Dt_dcCP_ClfDry'],
                               An_data_initial['An_CPIn'],
                               Fe_RUP,
                               Fe_RumMiCP,
                               Fe_CPend,
                               infusion_data['InfSI_NPNCPIn'],
                               coeff_dict)

########################################
# Step 7.2: Microbial Amino Acid Calculations
########################################
    AA_list = ['Arg', 'His', 'Ile', 'Leu',
               'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    # Dataframe for storing all individual amino acid values

    AA_values['Du_AAMic'] = nd.calculate_Du_AAMic(Du_MiTP_g,
                                                  AA_list,
                                                  nd.coeff_dict)
    AA_values['Du_IdAAMic'] = nd.calculate_Du_IdAAMic(AA_values['Du_AAMic'],
                                                      nd.coeff_dict)

    ########################################
    # Step 7.3: Complete diet_data and An_data
    ########################################
    diet_data = nd.calculate_diet_data_complete(diet_data_initial,
                                                animal_input['An_StatePhys'],
                                                Fe_CP,
                                                equation_selection['Monensin_eqn'],
                                                AA_values['Du_IdAAMic'],
                                                coeff_dict)
    An_data = nd.calculate_An_data_complete(An_data_initial,
                                            diet_data,
                                            animal_input['An_StatePhys'],
                                            Fe_CP,
                                            infusion_data,
                                            equation_selection['Monensin_eqn'],
                                            coeff_dict)

########################################
# Step 10: Metabolizable Protein Intake
########################################
    Du_idMiTP_g = nd.calculate_Du_idMiTP_g(Du_idMiCP_g,
                                           nd.coeff_dict)
    Du_idMiTP = nd.calculate_Du_idMiTP(Du_idMiTP_g)
    An_MPIn = nd.calculate_An_MPIn(diet_data['Dt_idRUPIn'],
                                   Du_idMiTP)
    An_MPIn_g = nd.calculate_An_MPIn_g(An_MPIn)

    ########################################
    # Step 8: Amino Acid Calculations
    ########################################

    AA_values['Abs_AA_g'] = nd.calculate_Abs_AA_g(AA_list,
                                                  An_data,
                                                  infusion_data,
                                                  infusion_data['Inf_Art'])

    AA_values['mPrtmx_AA'] = nd.calculate_mPrtmx_AA(AA_list,
                                                    coeff_dict)
    f_mPrt_max = nd.calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'],
                                         coeff_dict)
    AA_values['mPrtmx_AA2'] = nd.calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'],
                                                      f_mPrt_max)
    AA_values['AA_mPrtmx'] = nd.calculate_AA_mPrtmx(AA_list,
                                                    coeff_dict)
    AA_values['mPrt_AA_01'] = nd.calculate_mPrt_AA_01(AA_values['AA_mPrtmx'],
                                                      AA_list,
                                                      coeff_dict)
    AA_values['mPrt_k_AA'] = nd.calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'],
                                                    AA_values['mPrt_AA_01'],
                                                    AA_values['AA_mPrtmx'])

    Abs_EAA_g = nd.calculate_Abs_EAA_g(AA_values['Abs_AA_g'])
    Abs_neAA_g = nd.calculate_Abs_neAA_g(An_MPIn_g,
                                         Abs_EAA_g)
    Abs_OthAA_g = nd.calculate_Abs_OthAA_g(Abs_neAA_g,
                                           AA_values['Abs_AA_g'])
    Abs_EAA2b_g = nd.calculate_Abs_EAA2b_g(equation_selection['mPrt_eqn'],
                                           AA_values['Abs_AA_g'])
    mPrt_k_EAA2 = nd.calculate_mPrt_k_EAA2(AA_values.loc['Met', 'mPrtmx_AA2'],
                                           AA_values.loc['Met', 'mPrt_AA_01'],
                                           AA_values.loc['Met', 'AA_mPrtmx'])


    ########################################
    # Step 11: Milk Production Prediciton
    ########################################
    Mlk_NP_g = nd.calculate_Mlk_NP_g(animal_input['An_StatePhys'],
                                     animal_input['An_BW'],
                                     AA_values['Abs_AA_g'],
                                     AA_values['mPrt_k_AA'],
                                     Abs_neAA_g,
                                     Abs_OthAA_g,
                                     Abs_EAA2b_g,
                                     mPrt_k_EAA2,
                                     An_data['An_DigNDF'],
                                     An_data['An_DEInp'],
                                     An_data['An_DEStIn'],
                                     An_data['An_DEFAIn'],
                                     An_data['An_DErOMIn'],
                                     An_data['An_DENDFIn'],
                                     coeff_dict)
                                     
    Mlk_CP_g = nd.calculate_Mlk_CP_g(Mlk_NP_g)
        
    ########################################
    # Step 12: Body Frame and Reserve Gain
    ########################################
    Frm_Gain_empty = nd.calculate_Frm_Gain_empty(Frm_Gain,
                                                diet_data['Dt_DMIn_ClfLiq'],
                                                diet_data['Dt_DMIn_ClfStrt'], 
                                                coeff_dict)

    Body_Gain_empty = nd.calculate_Body_Gain_empty(Frm_Gain_empty, Rsrv_Gain_empty)

    An_REgain_Calf = nd.calculate_An_REgain_Calf(Body_Gain_empty, An_data['An_BW_empty'])

    Frm_NPgain = nd.calculate_Frm_NPgain(animal_input['An_StatePhys'],
                                        NPGain_FrmGain,
                                        Frm_Gain_empty,
                                        Body_Gain_empty,
                                        An_REgain_Calf)

    Body_NPgain = nd.calculate_Body_NPgain(Frm_NPgain, Rsrv_NPgain)
    Body_CPgain = nd.calculate_Body_CPgain(Body_NPgain, coeff_dict)
    Body_CPgain_g = nd.calculate_Body_CPgain_g(Body_CPgain)


########################################
# Step 13: Gestation Protein Use
########################################
    Gest_NCPgain_g = nd.calculate_Gest_NCPgain_g(GrUter_BWgain,coeff_dict)

    Gest_NPgain_g = nd.calculate_Gest_NPgain_g(Gest_NCPgain_g, coeff_dict)

    Gest_NPuse_g = nd.calculate_Gest_NPuse_g(Gest_NPgain_g, coeff_dict)


    Gest_CPuse_g = nd.calculate_Gest_CPuse_g(Gest_NPuse_g, coeff_dict)
    
########################################
# Step 14: Urinary Loss
########################################
    Ur_Nout_g = nd.calculate_Ur_Nout_g(diet_data['Dt_CPIn'],
                                    Fe_CP,
                                    Scrf_CP_g,
                                    Fe_CPend_g,
                                    Mlk_CP_g,
                                    Body_CPgain_g,
                                    Gest_CPuse_g)

    Ur_DEout = nd.calculate_Ur_DEout(Ur_Nout_g)

########################################
# Step 15: Energy Intake
########################################
    An_MEIn = nd.calculate_An_MEIn(animal_input['An_StatePhys'],
                                animal_input['An_BW'],
                                An_data['An_DEIn'],
                                An_data['An_GasEOut'],
                                Ur_DEout,
                                diet_data['Dt_DMIn_ClfLiq'],
                                diet_data['Dt_DEIn_base_ClfLiq'],
                                diet_data['Dt_DEIn_base_ClfDry'],
                                equation_selection['RumDevDisc_Clf'])
    An_NEIn = nd.calculate_An_NEIn(An_MEIn)
    An_NE = nd.calculate_An_NE(An_NEIn,An_data['An_DMIn'])

########################################
# Step 16: Energy Requirement
########################################
    # Maintenance Requirements
    An_NEmUse_NS = nd.calculate_An_NEmUse_NS(animal_input['An_StatePhys'],
                                             animal_input['An_BW'],
                                             An_data['An_BW_empty'],
                                             animal_input['An_Parity_rl'],
                                             diet_data['Dt_DMIn_ClfLiq'])
    
    An_NEm_Act_Graze = nd.calculate_An_NEm_Act_Graze(diet_data['Dt_PastIn'],
                                                     animal_input['DMI'],
                                                     diet_data['Dt_PastSupplIn'],
                                                     An_data['An_MBW'])

    An_NEm_Act_Parlor = nd.calculate_An_NEm_Act_Parlor(animal_input['An_BW'],
                                                       animal_input['Env_DistParlor'],
                                                       animal_input['Env_TripsParlor'])

    An_NEm_Act_Topo = nd.calculate_An_NEm_Act_Topo(animal_input['An_BW'],
                                                   animal_input['Env_Topo'])

    An_NEmUse_Act = nd.calculate_An_NEmUse_Act(An_NEm_Act_Graze,
                                               An_NEm_Act_Parlor,
                                               An_NEm_Act_Topo)

    An_NEmUse = nd.calculate_An_NEmUse(An_NEmUse_NS,
                                       An_NEmUse_Act,
                                       coeff_dict)

    An_MEmUse = nd.calculate_An_MEmUse(An_NEmUse,
                                       coeff_dict)

    # Gain Requirements
    Rsrv_Gain = nd.calculate_Rsrv_Gain(animal_input['Trg_RsrvGain'])

    Rsrv_Gain_empty = nd.calculate_Rsrv_Gain_empty(Rsrv_Gain)

    Rsrv_Fatgain = nd.calculate_Rsrv_Fatgain(Rsrv_Gain_empty,
                                             coeff_dict)

    CPGain_FrmGain = nd.calculate_CPGain_FrmGain(animal_input['An_BW'],
                                                 animal_input['An_BW_mature'])

    Rsrv_CPgain = nd.calculate_Rsrv_CPgain(CPGain_FrmGain,
                                           Rsrv_Gain_empty)

    Rsrv_NEgain = nd.calculate_Rsrv_NEgain(Rsrv_Fatgain,
                                           Rsrv_CPgain)

    Kr_ME_RE = nd.calculate_Kr_ME_RE(animal_input['Trg_MilkProd'],
                                     animal_input['Trg_RsrvGain'])

    Rsrv_MEgain = nd.calculate_Rsrv_MEgain(Rsrv_NEgain,
                                           Kr_ME_RE)

    FatGain_FrmGain = nd.calculate_FatGain_FrmGain(animal_input['An_StatePhys'],
                                                   An_REgain_Calf,
                                                   animal_input['An_BW'],
                                                   animal_input['An_BW_mature'])

    Frm_Gain = nd.calculate_Frm_Gain(animal_input['Trg_FrmGain'])

    Frm_Gain_empty = nd.calculate_Frm_Gain_empty(Frm_Gain,
                                                 diet_data['Dt_DMIn_ClfLiq'],
                                                 diet_data['Dt_DMIn_ClfStrt'],
                                                 coeff_dict)

    Frm_Fatgain = nd.calculate_Frm_Fatgain(FatGain_FrmGain,
                                           Frm_Gain_empty)

    NPGain_FrmGain = nd.calculate_NPGain_FrmGain(CPGain_FrmGain,
                                                 coeff_dict)

    Frm_NPgain = nd.calculate_Frm_NPgain(animal_input['An_StatePhys'],
                                         NPGain_FrmGain,
                                         Frm_Gain_empty,
                                         Body_Gain_empty,
                                         An_REgain_Calf)

    Frm_CPgain = nd.calculate_Frm_CPgain(Frm_NPgain,
                                         coeff_dict)

    Frm_NEgain = nd.calculate_Frm_NEgain(Frm_Fatgain,
                                         Frm_CPgain)

    Frm_MEgain = nd.calculate_Frm_MEgain(Frm_NEgain,
                                         coeff_dict)

    An_MEgain = nd.calculate_An_MEgain(Rsrv_MEgain,
                                       Frm_MEgain)

    # Gestation Requirement
    Gest_REgain = nd.calculate_Gest_REgain(GrUter_BWgain,
                                           coeff_dict)

    Gest_MEuse = nd.calculate_Gest_MEuse(Gest_REgain)

    # Milk Production Requirement
    Trg_NEmilk_Milk = nd.calculate_Trg_NEmilk_Milk(animal_input['Trg_MilkFatp'],
                                                   animal_input['Trg_MilkTPp'],
                                                   animal_input['Trg_MilkLacp'])

    Trg_Mlk_NEout = nd.calculate_Trg_Mlk_NEout(animal_input['Trg_MilkProd'],
                                               Trg_NEmilk_Milk)

    Trg_Mlk_MEout = nd.calculate_Trg_Mlk_MEout(Trg_Mlk_NEout,
                                               coeff_dict)

    # Total Metabolizalbe Energy Requirement
    Trg_MEuse = nd.calculate_Trg_MEuse(An_MEmUse,
                                       An_MEgain,
                                       Gest_MEuse,
                                       Trg_Mlk_MEout)

    ########################################
    # Step 17: Protein Requirement
    ########################################
    # Maintenance Requirement
    Fe_NPend = nd.calculate_Fe_NPend(Fe_CPend)
    
    Fe_NPend_g = nd.calculate_Fe_NPend_g(Fe_NPend)
    
    Fe_MPendUse_g_Trg = nd.calculate_Fe_MPendUse_g_Trg(animal_input['An_StatePhys'],
                                                       Fe_CPend_g,
                                                       Fe_NPend_g,
                                                       coeff_dict)
       
    Scrf_NP_g = nd.calculate_Scrf_NP_g(Scrf_CP_g,
                                       coeff_dict)
        
    Scrf_MPUse_g_Trg = nd.calculate_Scrf_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                     Scrf_CP_g,
                                                     Scrf_NP_g,
                                                     coeff_dict)
        
    Ur_Nend_g = nd.calculate_Ur_Nend_g(animal_input['An_BW'])
    
    Ur_NPend_g = nd.calculate_Ur_NPend_g(Ur_Nend_g)
    
    Ur_MPendUse_g = nd.calculate_Ur_MPendUse_g(Ur_NPend_g)
    
    An_MPm_g_Trg = nd.calculate_An_MPm_g_Trg(Fe_MPendUse_g_Trg,
                                             Scrf_MPUse_g_Trg,
                                             Ur_MPendUse_g)
    
    # Gain Requirement
    Body_NPgain_g = nd.calculate_Body_NPgain_g(Body_NPgain)
    
    Body_MPUse_g_Trg = nd.calculate_Body_MPUse_g_Trg_initial(Body_NPgain_g,
                                                             coeff_dict)

    # Gestation Requirement
    Gest_MPUse_g_Trg = nd.calculate_Gest_MPUse_g_Trg(Gest_NPuse_g,
                                                     coeff_dict)
    
    # Milk Requirement
    Trg_Mlk_NP_g = nd.calculate_Trg_Mlk_NP_g(animal_input['Trg_MilkProd'],
                                             animal_input['Trg_MilkTPp'])
    
    Mlk_MPUse_g_Trg = nd.calculate_Mlk_MPUse_g_Trg(Trg_Mlk_NP_g,
                                                    coeff_dict)

    # Total Protein Requirement
    # NOTE This initial protein requirement is the final value in most cases. There is an
    #      adjustment made when An_StatePhys == "Heifer" and Diff_MPuse_g > 0
    #      Some of the values used to make this adjustment are used elsewhere so it can't just 
    #      be put behind an if statement 
    An_MPuse_g_Trg = nd.calculate_An_MPuse_g_Trg_initial(An_MPm_g_Trg,
                                                         Body_MPUse_g_Trg,
                                                         Gest_MPUse_g_Trg,
                                                         Mlk_MPUse_g_Trg)
    
    An_MEIn_approx = nd.calculate_An_MEIn_approx(An_data['An_DEInp'],
                                                 An_data['An_DENPNCPIn'],
                                                 An_data['An_DigTPaIn'],
                                                 Body_NPgain,
                                                 An_data['An_GasEOut'],
                                                 coeff_dict)
    
    Min_MPuse_g = nd.calculate_Min_MPuse_g(animal_input['An_StatePhys'],
                                           An_MPuse_g_Trg,
                                           animal_input['An_BW'],
                                           animal_input['An_BW_mature'],
                                           An_MEIn_approx)
    
    Diff_MPuse_g = nd.calculate_Diff_MPuse_g(Min_MPuse_g,
                                             An_MPuse_g_Trg)
    
    Frm_NPgain_g = nd.calculate_Frm_NPgain_g(Frm_NPgain)
    
    An_BWmature_empty = nd.calculate_An_BWmature_empty(animal_input['An_BW_mature'],
                                                       coeff_dict)
    

    Kg_MP_NP_Trg = nd.calculate_Kg_MP_NP_Trg(animal_input['An_StatePhys'],
                                             animal_input['An_Parity_rl'],
                                             animal_input['An_BW'],
                                             An_data['An_BW_empty'],
                                             animal_input['An_BW_mature'],
                                             An_BWmature_empty,
                                             MP_NP_efficiency_input,
                                             coeff_dict)

    Frm_MPUse_g_Trg = nd.calculate_Frm_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                   Frm_NPgain_g,
                                                   Kg_MP_NP_Trg,
                                                   Diff_MPuse_g)
            
    Rsrv_NPgain_g = nd.calculate_Rsrv_NPgain_g(Rsrv_NPgain)
    
    Rsrv_MPUse_g_Trg = nd.calculate_Rsrv_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                     Diff_MPuse_g,
                                                     Rsrv_NPgain_g,
                                                     Kg_MP_NP_Trg)
    
    # Recalculate 
    Body_MPUse_g_Trg = nd.calculate_Body_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                     Diff_MPuse_g,
                                                     Body_NPgain_g,
                                                     Body_MPUse_g_Trg,
                                                     Kg_MP_NP_Trg)
    # Recalculate
    An_MPuse_g_Trg = nd.calculate_An_MPuse_g_Trg(An_MPm_g_Trg,
                                                 Frm_MPUse_g_Trg,
                                                 Rsrv_MPUse_g_Trg,
                                                 Gest_MPUse_g_Trg,
                                                 Mlk_MPUse_g_Trg)
    
    ########################################
    # Milk Production Calculations
    ########################################
    An_LactDay_MlkPred = nd.calculate_An_LactDay_MlkPred(animal_input['An_LactDay'])

    Trg_Mlk_Fat = nd.calculate_Trg_Mlk_Fat(animal_input['Trg_MilkProd'],
                                        animal_input['Trg_MilkFatp'])

    Trg_Mlk_Fat_g = nd.calculate_Trg_Mlk_Fat_g(Trg_Mlk_Fat)

    Mlk_Fatemp_g = nd.calculate_Mlk_Fatemp_g(animal_input['An_StatePhys'],
                                          An_LactDay_MlkPred,
                                          animal_input['DMI'],
                                          diet_data['Dt_FAIn'],
                                          diet_data['Dt_DigC160In'],
                                          diet_data['Dt_DigC183In'],
                                          AA_values.loc['Ile', 'Abs_AA_g'],
                                          AA_values.loc['Met', 'Abs_AA_g'])

    Mlk_Fat_g = nd.calculate_Mlk_Fat_g(equation_selection['mFat_eqn'],
                                    Trg_Mlk_Fat_g,
                                    Mlk_Fatemp_g)

    Mlk_Fat = nd.calculate_Mlk_Fat(Mlk_Fat_g)

    Mlk_NP = nd.calculate_Mlk_NP(Mlk_NP_g)

    Mlk_Prod_comp = nd.calculate_Mlk_Prod_comp(animal_input['An_Breed'],
                                            Mlk_NP,
                                            Mlk_Fat,
                                            An_data['An_DEIn'],
                                            An_LactDay_MlkPred,
                                            animal_input['An_Parity_rl'])

    An_MPavail_Milk_Trg = nd.calculate_An_MPavail_Milk_Trg(An_MPIn,
                                                        An_MPuse_g_Trg,
                                                        Mlk_MPUse_g_Trg)    

    Mlk_NP_MPalow_Trg_g = nd.calculate_Mlk_NP_MPalow_Trg_g(An_MPavail_Milk_Trg,
                                                        coeff_dict)

    Mlk_Prod_MPalow = nd.calculate_Mlk_Prod_MPalow(Mlk_NP_MPalow_Trg_g,
                                                animal_input['Trg_MilkTPp'])

    An_MEavail_Milk = nd.calculate_An_MEavail_Milk(An_MEIn,
                                                An_MEgain,
                                                An_MEmUse,
                                                Gest_MEuse)

    Mlk_Prod_NEalow = nd.calculate_Mlk_Prod_NEalow(An_MEavail_Milk,
                                                Trg_NEmilk_Milk,
                                                coeff_dict)

    Mlk_Prod = nd.calculate_Mlk_Prod(animal_input['An_StatePhys'],
                                  equation_selection['mProd_eqn'],
                                  Mlk_Prod_comp,
                                  Mlk_Prod_NEalow,
                                  Mlk_Prod_MPalow,
                                  animal_input['Trg_MilkProd'])

    MlkNP_Milk = nd.calculate_MlkNP_Milk(animal_input['An_StatePhys'],
                                      Mlk_NP_g,
                                      Mlk_Prod)
    
    MlkFat_Milk = nd.calculate_MlkFat_Milk(animal_input['An_StatePhys'],
                                        Mlk_Fat,
                                        Mlk_Prod)
    
    MlkNE_Milk = nd.calculate_MlkNE_Milk(MlkFat_Milk,
                                      MlkNP_Milk,
                                      animal_input['Trg_MilkLacp'])
    
    Mlk_NEout = nd.calculate_Mlk_NEout(MlkNE_Milk,
                                    Mlk_Prod)

    Mlk_MEout = nd.calculate_Mlk_MEout(Mlk_NEout,
                                    nd.coeff_dict)

    ########################################
    # Mineral Requirement Calculations
    ########################################
    Body_Gain = nd.calculate_Body_Gain(Frm_Gain,
                                    Rsrv_Gain)

    ### Calcium ###
    Ca_Mlk = nd.calculate_Ca_Mlk(animal_input['An_Breed'])

    Fe_Ca_m = nd.calculate_Fe_Ca_m(An_data['An_DMIn'])

    An_Ca_g = nd.calculate_An_Ca_g(animal_input['An_BW_mature'],
                                animal_input['An_BW'],
                                Body_Gain)

    An_Ca_y = nd.calculate_An_Ca_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])

    An_Ca_l = nd.calculate_An_Ca_l(Mlk_NP_g,
                                Ca_Mlk,
                                animal_input['Trg_MilkProd'],
                                animal_input['Trg_MilkTPp'])

    An_Ca_Clf = nd.calculate_An_Ca_Clf(An_data['An_BW_empty'],
                                    Body_Gain_empty)

    An_Ca_req = nd.calculate_An_Ca_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Ca_Clf,
                                    Fe_Ca_m,
                                    An_Ca_g,
                                    An_Ca_y,
                                    An_Ca_l)

    An_Ca_bal = nd.calculate_An_Ca_bal(diet_data['Abs_CaIn'],
                                    An_Ca_req)

    An_Ca_prod = nd.calculate_An_Ca_prod(An_Ca_y,
                                      An_Ca_l,
                                      An_Ca_g)

    ### Phosphorus ###
    Ur_P_m = nd.calculate_Ur_P_m(animal_input['An_BW']) 

    Fe_P_m = nd.calculate_Fe_P_m(animal_input['An_Parity_rl'],
                              An_data['An_DMIn'])

    An_P_m = nd.calculate_An_P_m(Ur_P_m,
                              Fe_P_m)

    An_P_g = nd.calculate_An_P_g(animal_input['An_BW_mature'],
                              animal_input['An_BW'],
                              Body_Gain)

    An_P_y = nd.calculate_An_P_y(animal_input['An_GestDay'],
                              animal_input['An_BW'])

    An_P_l = nd.calculate_An_P_l(animal_input['Trg_MilkProd'],
                              MlkNP_Milk)

    An_P_Clf = nd.calculate_An_P_Clf(An_data['An_BW_empty'],
                                  Body_Gain_empty)    

    An_P_req = nd.calculate_An_P_req(animal_input['An_StatePhys'],
                                  diet_data['Dt_DMIn_ClfLiq'],
                                  An_P_Clf,
                                  An_P_m,
                                  An_P_g,
                                  An_P_y,
                                  An_P_l)

    An_P_bal = nd.calculate_An_P_bal(diet_data['Abs_PIn'],
                                  An_P_req)

    Fe_P_g = nd.calculate_Fe_P_g(diet_data['Dt_PIn'],
                              An_P_l,
                              An_P_y,
                              An_P_g,
                              Ur_P_m)

    An_P_prod = nd.calculate_An_P_prod(An_P_y,
                                    An_P_l,
                                    An_P_g)

    ### Magnesium ###
    An_Mg_Clf = nd.calculate_An_Mg_Clf(An_data['An_BW_empty'],
                                    Body_Gain_empty)

    Ur_Mg_m = nd.calculate_Ur_Mg_m(animal_input['An_BW'])

    Fe_Mg_m = nd.calculate_Fe_Mg_m(An_data['An_DMIn'])

    An_Mg_m = nd.calculate_An_Mg_m(Ur_Mg_m,
                                Fe_Mg_m)

    An_Mg_g = nd.calculate_An_Mg_g(Body_Gain)

    An_Mg_y = nd.calculate_An_Mg_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])

    An_Mg_l = nd.calculate_An_Mg_l(animal_input['Trg_MilkProd'])

    An_Mg_req = nd.calculate_An_Mg_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Mg_Clf,
                                    An_Mg_m,
                                    An_Mg_g,
                                    An_Mg_y,
                                    An_Mg_l)

    An_Mg_bal = nd.calculate_An_Mg_bal(diet_data['Abs_MgIn'],
                                    An_Mg_req)

    An_Mg_prod = nd.calculate_An_Mg_prod(An_Mg_y,
                                      An_Mg_l,
                                      An_Mg_g)

    ### Sodium ###
    An_Na_Clf = nd.calculate_An_Na_Clf(An_data['An_BW_empty'],
                                    Body_Gain_empty)

    Fe_Na_m = nd.calculate_Fe_Na_m(An_data['An_DMIn'])

    An_Na_g = nd.calculate_An_Na_g(Body_Gain)

    An_Na_y = nd.calculate_An_Na_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])

    An_Na_l = nd.calculate_An_Na_l(animal_input['Trg_MilkProd'])

    An_Na_req = nd.calculate_An_Na_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Na_Clf,
                                    Fe_Na_m,
                                    An_Na_g,
                                    An_Na_y,
                                    An_Na_l)

    An_Na_bal = nd.calculate_An_Na_bal(diet_data['Abs_NaIn'],
                                    An_Na_req)

    An_Na_prod = nd.calculate_An_Na_prod(An_Na_y,
                                      An_Na_l,
                                      An_Na_g)

    ### Chlorine ###
    An_Cl_Clf = nd.calculate_An_Cl_Clf(An_data['An_BW_empty'],
                                    Body_Gain_empty)
    
    Fe_Cl_m = nd.calculate_Fe_Cl_m(An_data['An_DMIn'])

    An_Cl_g = nd.calculate_An_Cl_g(Body_Gain)

    An_Cl_y = nd.calculate_An_Cl_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])

    An_Cl_l = nd.calculate_An_Cl_l(animal_input['Trg_MilkProd'])

    An_Cl_req = nd.calculate_An_Cl_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Cl_Clf,
                                    Fe_Cl_m,
                                    An_Cl_g,
                                    An_Cl_y,
                                    An_Cl_l)

    An_Cl_bal = nd.calculate_An_Cl_bal(diet_data['Abs_ClIn'],
                                    An_Cl_req)

    An_Cl_prod = nd.calculate_An_Cl_prod(An_Cl_y,
                                      An_Cl_l,
                                      An_Cl_g)

    ### Potassium ###
    An_K_Clf = nd.calculate_An_K_Clf(An_data['An_BW_empty'],
                                  Body_Gain_empty)

    Ur_K_m = nd.calculate_Ur_K_m(animal_input['Trg_MilkProd'],
                              animal_input['An_BW'])

    Fe_K_m = nd.calculate_Fe_K_m(An_data['An_DMIn'])

    An_K_m = nd.calculate_An_K_m(Ur_K_m,
                              Fe_K_m)

    An_K_g = nd.calculate_An_K_g(Body_Gain)

    An_K_y = nd.calculate_An_K_y(animal_input['An_GestDay'],
                              animal_input['An_BW'])

    An_K_l = nd.calculate_An_K_l(animal_input['Trg_MilkProd'])

    An_K_req = nd.calculate_An_K_req(animal_input['An_StatePhys'],
                                  diet_data['Dt_DMIn_ClfLiq'],
                                  An_K_Clf,
                                  An_K_m,
                                  An_K_g,
                                  An_K_y,
                                  An_K_l)

    An_K_bal = nd.calculate_An_K_bal(diet_data['Abs_KIn'],
                                  An_K_req)

    An_K_prod = nd.calculate_An_K_prod(An_K_y,
                                    An_K_l,
                                    An_K_g)

    ### Sulphur ###
    An_S_req = nd.calculate_An_S_req(An_data['An_DMIn'])

    An_S_bal = nd.calculate_An_S_bal(diet_data['Dt_SIn'],
                                  An_S_req)

    ### Cobalt ###
    An_Co_req = nd.calculate_An_Co_req(An_data['An_DMIn'])

    An_Co_bal = nd.calculate_An_Co_bal(diet_data['Abs_CoIn'],
                                    An_Co_req)

    ### Copper ###
    An_Cu_Clf = nd.calculate_An_Cu_Clf(animal_input['An_BW'],
                                    Body_Gain_empty)

    An_Cu_m = nd.calculate_An_Cu_m(animal_input['An_BW'])

    An_Cu_g = nd.calculate_An_Cu_g(Body_Gain)

    An_Cu_y = nd.calculate_An_Cu_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])

    An_Cu_l = nd.calculate_An_Cu_l(animal_input['Trg_MilkProd'])

    An_Cu_req = nd.calculate_An_Cu_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Cu_Clf,
                                    An_Cu_m,
                                    An_Cu_g,
                                    An_Cu_y,
                                    An_Cu_l)

    An_Cu_bal = nd.calculate_An_Cu_bal(diet_data['Abs_CuIn'],
                                    An_Cu_req)

    An_Cu_prod = nd.calculate_An_Cu_prod(An_Cu_y,
                                      An_Cu_l,
                                      An_Cu_g)

    ### Iodine ###
    An_I_req = nd.calculate_An_I_req(animal_input['An_StatePhys'],
                                  An_data['An_DMIn'],
                                  animal_input['An_BW'],
                                  animal_input['Trg_MilkProd'])

    An_I_bal = nd.calculate_An_I_bal(diet_data['Dt_IIn'],
                                  An_I_req)

    ### Iron ###
    An_Fe_Clf = nd.calculate_An_Fe_Clf(Body_Gain)

    An_Fe_g = nd.calculate_An_Fe_g(Body_Gain)

    An_Fe_y = nd.calculate_An_Fe_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])

    An_Fe_l = nd.calculate_An_Fe_l(animal_input['Trg_MilkProd'])

    An_Fe_req = nd.calculate_An_Fe_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Fe_Clf,
                                    An_Fe_g,
                                    An_Fe_y,
                                    An_Fe_l)

    An_Fe_bal = nd.calculate_An_Fe_bal(diet_data['Abs_FeIn'],
                                    An_Fe_req)

    An_Fe_prod = nd.calculate_An_Fe_prod(An_Fe_y,
                                      An_Fe_l,
                                      An_Fe_g)

    ### Maganese ###
    An_Mn_Clf = nd.calculate_An_Mn_Clf(animal_input['An_BW'],
                                    Body_Gain)

    An_Mn_m = nd.calculate_An_Mn_m(animal_input['An_BW'])
    
    An_Mn_g = nd.calculate_An_Mn_g(Body_Gain)
    
    An_Mn_y = nd.calculate_An_Mn_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])
    
    An_Mn_l = nd.calculate_An_Mn_l(animal_input['Trg_MilkProd'])
    
    An_Mn_req = nd.calculate_An_Mn_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Mn_Clf,
                                    An_Mn_m,
                                    An_Mn_g,
                                    An_Mn_y,
                                    An_Mn_l)
    
    An_Mn_bal = nd.calculate_An_Mn_bal(diet_data['Abs_MnIn'],
                                    An_Mn_req)
    
    An_Mn_prod = nd.calculate_An_Mn_prod(An_Mn_y,
                                      An_Mn_l,
                                      An_Mn_g)

    ### Selenium ###  
    An_Se_req = nd.calculate_An_Se_req(An_data['An_DMIn'])

    An_Se_bal = nd.calculate_An_Se_bal(diet_data['Dt_SeIn'],
                                    An_Se_req)

    ### Zinc ###
    An_Zn_Clf = nd.calculate_An_Zn_Clf(An_data['An_DMIn'],
                                    Body_Gain)

    An_Zn_m = nd.calculate_An_Zn_m(An_data['An_DMIn'])

    An_Zn_g = nd.calculate_An_Zn_g(Body_Gain)

    An_Zn_y = nd.calculate_An_Zn_y(animal_input['An_GestDay'],
                                animal_input['An_BW'])

    An_Zn_l = nd.calculate_An_Zn_l(animal_input['Trg_MilkProd'])

    An_Zn_req = nd.calculate_An_Zn_req(animal_input['An_StatePhys'],
                                    diet_data['Dt_DMIn_ClfLiq'],
                                    An_Zn_Clf,
                                    An_Zn_m,
                                    An_Zn_g,
                                    An_Zn_y,
                                    An_Zn_l)

    An_Zn_bal = nd.calculate_An_Zn_bal(diet_data['Abs_ZnIn'],
                                    An_Zn_req)

    An_Zn_prod = nd.calculate_An_Zn_prod(An_Zn_y, 
                                      An_Zn_l,
                                      An_Zn_g)

    ### DCAD ###
    An_DCADmeq = nd.calculate_An_DCADmeq(diet_data['Dt_K'],
                                      diet_data['Dt_Na'],
                                      diet_data['Dt_Cl'],
                                      diet_data['Dt_S'])

    ########################################
    # Temporary results export
    ########################################
    # print(Trg_MEuse)
    
    locals_dict = locals()
    test_output = ModelOutput(locals_input=locals_dict)
    

    model_out_dict = {
        'infusion_data' : infusion_data,
        'Du_MiN_g' : Du_MiN_g,
        # testing Monensin:
        'Dt_DEIn': diet_data['Dt_DEIn'],
        'An_DEIn': An_data['An_DEIn'],
        'Trg_NEmilk_Milk' : Trg_NEmilk_Milk 
    }

    return animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict, test_output

### RUN MODEL ###
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict, test_output = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)


# Testing ModelOutput Class

In [67]:
# Access a category
pprint(test_output.Inputs)


{'MP_NP_efficiency_input': {'Trg_AbsHis_NPHis': 0.75,
                            'Trg_AbsIle_NPIle': 0.71,
                            'Trg_AbsLeu_NPLeu': 0.73,
                            'Trg_AbsLys_NPLys': 0.72,
                            'Trg_AbsMet_NPMet': 0.73,
                            'Trg_AbsPhe_NPPhe': 0.6,
                            'Trg_AbsThr_NPThr': 0.64,
                            'Trg_AbsTrp_NPTrp': 0.86,
                            'Trg_AbsVal_NPVal': 0.74,
                            'Trg_MP_NP': 0.69},
 'animal_input': {'An_305RHA_MlkTP': 280.0,
                  'An_AgeDay': 820.8,
                  'An_AgeDryFdStart': 14.0,
                  'An_BCS': 3.0,
                  'An_BW': 624.795,
                  'An_BW_mature': 700.0,
                  'An_Breed': 'Holstein',
                  'An_GestDay': 46.0,
                  'An_GestLength': 280.0,
                  'An_LactDay': 100.0,
                  'An_Parity_rl': 1.0,
                  'An_PrePartDa

In [68]:
# Access an entry within a category
pprint(test_output.Inputs['animal_input'])


{'An_305RHA_MlkTP': 280.0,
 'An_AgeDay': 820.8,
 'An_AgeDryFdStart': 14.0,
 'An_BCS': 3.0,
 'An_BW': 624.795,
 'An_BW_mature': 700.0,
 'An_Breed': 'Holstein',
 'An_GestDay': 46.0,
 'An_GestLength': 280.0,
 'An_LactDay': 100.0,
 'An_Parity_rl': 1.0,
 'An_PrePartDay': -234.0,
 'An_PrePartWk': -33.42857142857143,
 'An_StatePhys': 'Lactating Cow',
 'DMI': 24.521,
 'Env_DistParlor': 0.0,
 'Env_TempCurr': 22.0,
 'Env_Topo': 0.0,
 'Env_TripsParlor': 0.0,
 'Fet_BWbrth': 44.1,
 'Trg_FrmGain': 0.19,
 'Trg_MilkFatp': 4.55,
 'Trg_MilkLacp': 4.85,
 'Trg_MilkProd': 25.062,
 'Trg_MilkTPp': 3.66,
 'Trg_RsrvGain': 0.0}


In [69]:
# Access an individual value from an entry (dictionary)
test_output.Inputs['animal_input']['An_BW']


624.795

In [70]:
# Access an individual value from an entry (dataframe)
pprint(test_output.Inputs['user_diet'].loc[0, 'kg_user'])

# Access a column of a dataframe 
pprint(test_output.Inputs['user_diet'].loc[:, 'kg_user'])


8.210156
0    8.210156
1    6.732329
2    5.473438
3    4.105078
Name: kg_user, dtype: float32


# Test Execute Model

In [1]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint
from icecream import ic
# use package dir to always use /data folder regardless of where this is executed from
import importlib_resources

path_to_package_data = importlib_resources.files("nasem_dairy.data")

# Read_csv to load required data into env
user_diet_in, animal_input_in, equation_selection_in = nd.read_csv_input(path_to_package_data.joinpath("./input.csv"))

# Load feed library
feed_library_in = pd.read_csv(path_to_package_data.joinpath("NASEM_feed_library.csv"))

# Prepare infusion data (This will be optional as a default dict of 0 is provided to function otherwise)
infusion_custom = nd.read_infusion_input(path_to_package_data.joinpath("infusion_input.csv"))


In [2]:
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)


# Run model - different options

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
An_data

## Toggle infusions
Notes: found that infusions are being imported within the function, need to move all file imports out of the run model function.
There is also double up of the logic for use_infusions, so modifying that to check which one to use once. 
It could actually be avoided by just providing a default infusion dictionary if one is not given to function directly.
These changes make the 'use_infusions' flag obsolete

In [5]:
# Test DMI equations
equation_selection_in['use_infusions'] = 0 # FALSE

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
# infusion_data


In [6]:
# equation_selection_in['use_infusions'] = 1

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict,
    infusion_input= infusion_custom)

# diet_info
model_out_dict

{'infusion_data': {'Inf_Acet_g': 3000.0,
  'Inf_ADF_g': 4000.0,
  'Inf_Arg_g': 5000.0,
  'Inf_Ash_g': 6000.0,
  'Inf_Butr_g': 7000.0,
  'Inf_CP_g': 8000.0,
  'Inf_CPARum_CP': 9000.0,
  'Inf_CPBRum_CP': 10000.0,
  'Inf_CPCRum_CP': 11000.0,
  'Inf_dcFA': 12000.0,
  'Inf_dcRUP': 13000.0,
  'Inf_DM_g': 14000.0,
  'Inf_EE_g': 15000.0,
  'Inf_FA_g': 16000.0,
  'Inf_Glc_g': 17000.0,
  'Inf_His_g': 18000.0,
  'Inf_Ile_g': 19000.0,
  'Inf_KdCPB': 20000.0,
  'Inf_Leu_g': 21000.0,
  'Inf_Lys_g': 22000.0,
  'Inf_Met_g': 23000.0,
  'Inf_NDF_g': 24000.0,
  'Inf_NPNCP_g': 25000.0,
  'Inf_Phe_g': 26000.0,
  'Inf_Prop_g': 27000.0,
  'Inf_St_g': 28000.0,
  'Inf_Thr_g': 29000.0,
  'Inf_Trp_g': 30000.0,
  'Inf_ttdcSt': 31000.0,
  'Inf_Val_g': 32000.0,
  'Inf_VFA_g': 33000.0,
  'Inf_Location': nan,
  'Inf_DMIn': 14.0,
  'Inf_StIn': 28.0,
  'Inf_NDFIn': 24.0,
  'Inf_ADFIn': 4.0,
  'Inf_GlcIn': 17.0,
  'Inf_CPIn': 8.0,
  'Inf_NPNCPIn': 25.0,
  'Inf_FAIn': 16.0,
  'Inf_AshIn': 6.0,
  'Inf_VFAIn': 33.0,
  'Inf

## Test DMI equation selections

In [7]:
# Equation = 0 should match user input:
equation_selection_in['DMIn_eqn'] = 0
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

24.521

In [8]:
# Equation = 8 should calculate calculate_Dt_DMIn_Lact1()
equation_selection_in['DMIn_eqn'] = 8
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

21.687577670699824

In [9]:
# make function to test all values
def iterate_test_DMI_eqns(DMIn_eqn):
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['DMIn_eqn'] = DMIn_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (animal_input['DMI'])

[iterate_test_DMI_eqns(DMIn_eqn_value) for DMIn_eqn_value in range(0, 16)]


DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: 1
DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: 9


[24.521,
 24.521,
 11.504513756424398,
 16.842879099441184,
 11.474631902511609,
 12.805095198359778,
 10.866105707801568,
 12.251816154227694,
 21.687577670699824,
 24.521,
 12.323872879290622,
 12.364693050000001,
 11.504513756424398,
 16.842879099441184,
 11.474631902511609,
 12.805095198359778]

In [ ]:
## Still need to test this condition: 
#if animal_input['An_PrePartWk'] > An_PrePartWkDurat:

print(animal_input['An_PrePartWk'])

# Calculate additional physiology values
An_PrePartDay = animal_input_in['An_GestDay'] - animal_input_in['An_GestLength']
An_PrePartDay / 7
# animal_input_in['An_PrePartWk']

# Test equation_selection['MiN_eqn']

In [10]:
# equation_selection['MiN_eqn']


# make function to test all values
def iterate_test_Du_MiN_g_eqns(MiN_eqn):
    print(MiN_eqn)
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['MiN_eqn'] = MiN_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (model_out_dict['Du_MiN_g'])

[iterate_test_Du_MiN_g_eqns(DMIn_eqn_value) for DMIn_eqn_value in range(1, 4)]

1
2
3


[299.31825131136395, 122.74827303321794, 276.5276811382165]

In [11]:
equation_selection_in

{'Use_DNDF_IV': 0.0,
 'DMIn_eqn': 8,
 'mProd_eqn': 0.0,
 'MiN_eqn': 1.0,
 'use_infusions': 0,
 'NonMilkCP_ClfLiq': 0.0,
 'Monensin_eqn': 0.0,
 'mPrt_eqn': 0.0}

# test Monensin eqn

In [12]:
equation_selection_in


{'Use_DNDF_IV': 0.0,
 'DMIn_eqn': 8,
 'mProd_eqn': 0.0,
 'MiN_eqn': 1.0,
 'use_infusions': 0,
 'NonMilkCP_ClfLiq': 0.0,
 'Monensin_eqn': 0.0,
 'mPrt_eqn': 0.0}

In [13]:
# make function to test all values
def iterate_test_Mon_eqns(Monensin_eqn):
    print(Monensin_eqn)
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['Monensin_eqn'] = Monensin_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = nd.execute_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (model_out_dict['Dt_DEIn'], model_out_dict['An_DEIn'])

[iterate_test_Mon_eqns(Monensin_eqn) for Monensin_eqn in range(0, 2)]

0
1


[(array(69.04770702), array(69.04770702)),
 (array(70.42866116), array(70.42866116))]